# Homework 2

In [ ]:
!pip install pyperplan

# !!! Warning !!!
# Colab only supports sympy version 1.10, but our catsoop server runs the latest sympy.
# The difference between the two versions should be minimal and should not be noticeable.
# However, if you passed the tests on Colab, but catsoop raises errors, 
# please let us know (e.g., via private piazza question), and we will be happy to help.
!pip install sympy==1.10


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import numpy as np
import sympy



## Three room erratic vacumm world


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

from abc import ABC, abstractmethod


class Graph(object):
    """A graph connects nodes (vertices) by edges (links).

    Each edge can also
    have a length associated with it. The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C. You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more links with g.connect('B', 'C', 3), then
    inverse link is also added. You can use g.nodes() to get a list of nodes,
    g.get('A') to get a dict of links out of A, and g.get('A', 'B') to get the
    length of the link from A to B. 'Lengths' can actually be any object at
    all, and nodes can be any hashable object.
    """

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric
        edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, a, b, distance=1):
        """Add a link from a and b of given distance, and also add the inverse
        link if the graph is undirected."""
        self.connect1(a, b, distance)
        if not self.directed:
            self.connect1(b, a, distance)

    def connect1(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        """Return a link distance or a dict of {node: distance} entries.

        .get(a,b) returns the distance or None; .get(a) returns a dict
        of {node: distance} entries, possibly {}.
        """
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


class Problem(object):
    """The abstract class for a formal problem, based on AIMA.

    You should subclass this and implement abstract methods. Then you
    will create instances of your subclass and solve them with the
    various search functions.
    """

    def __init__(self, initial):
        self.initial = initial

    @abstractmethod
    def actions(self, state):
        """Returns the allowed actions in a given state.

        The result would typically be a list. But if there are many
        actions, consider yielding them one at a time in an iterator,
        rather than building them all at once.
        """
        ...

    @abstractmethod
    def step(self, state, action):
        """Returns the next state when executing a given action in a given
        state.

        The action must be one of self.actions(state).
        """
        ...

    @abstractmethod
    def goal_test(self, state):
        """Checks if the state is a goal."""
        ...


""" [AIMA Figure 4.9]
Eight possible states of the vacumm world
Each state is represented as
   *       "State of the left room"      "State of the right room"   "Room in which the agent
                                                                      is present"
1 - DDL     Dirty                         Dirty                       Left
2 - DDR     Dirty                         Dirty                       Right
3 - DCL     Dirty                         Clean                       Left
4 - DCR     Dirty                         Clean                       Right
5 - CDL     Clean                         Dirty                       Left
6 - CDR     Clean                         Dirty                       Right
7 - CCL     Clean                         Clean                       Left
8 - CCR     Clean                         Clean                       Right
"""
vacuum_world = Graph(
    dict(DDL=dict(Suck=['CCL', 'CDL'], Right=['DDR']),
         DDR=dict(Suck=['CCR', 'DCR'], Left=['DDR']),
         DCL=dict(Suck=['CCL'], Right=['DCR']),
         DCR=dict(Suck=['DCR', 'DDR'], Left=['DCL']),
         CDL=dict(Suck=['CDL', 'DDL'], Right=['CDR']),
         CDR=dict(Suck=['CCR'], Left=['CDL']),
         CCL=dict(Suck=['CCL', 'DCL'], Right=['CCR']),
         CCR=dict(Suck=['CCR', 'CDR'], Left=['CCL'])))


class GraphProblemNonDet(Problem):
    """A graph problem where an action can lead to nondeterministic output i.e.
    multiple possible states."""

    def __init__(self, initial, goal, graph):
        super().__init__(initial)
        self.goal = goal  # a set of states
        self.graph = graph

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get(A).keys())

    def step(self, state, action):
        return self.graph.get(state, action)

    def goal_test(self, state):
        return state in self.goal


def AO_DFS(problem):
    """Used when the environment is nondeterministic and completely observable.

    Contains OR nodes where the agent is free to choose any action.
    After every action there is an AND node which contains all possible
    states the agent may reach due to stochastic nature of environment.
    The agent must be able to handle all possible states of the AND node
    (as it may end up in any of them). Returns a conditional plan to
    reach goal state, or failure if the former is not possible.
    """

    # functions used by and_or_search
    def or_search(state, path, problem):
        """returns a plan as a list of actions."""
        if problem.goal_test(state):
            return []
        if state in path:
            return None
        for action in problem.actions(state):
            plan_dict = and_search(problem.step(state, action), path + [
                state,
            ], problem)
            if plan_dict is not None:
                return [action, plan_dict]

    def and_search(states, path, problem):
        """Returns plan in form of dictionary where we take action plan[s] if
        we reach state s."""
        plan_dict = {}
        for s in states:
            plan = or_search(s, path, problem)
            if plan is None:
                return None
            plan_dict[s] = plan
        return plan_dict

    # body of and or search
    return or_search(problem.initial, [], problem)


def sort_graph_dict(data):
    """Normalizes a graph_dict."""
    if isinstance(data, dict):
        for key in data:
            data[key] = sort_graph_dict(data[key])
        return data
    if isinstance(data, list) or isinstance(data, tuple):
        return sorted(data)
    elif isinstance(data, str):
        return data
    else:
        raise ValueError(f"Type {type(data)} not supported")


def graph_dict_md5(data) -> str:
    """Turn Python object into md5 hash.

    Handles nested standard Python types. Used for testing.
    """
    import pprint, hashlib
    return hashlib.md5(pprint.pformat(
        sort_graph_dict(data)).encode('utf-8')).hexdigest()


# A problem that can be passed to AO_DFS; uncomment if you want to use it for testing on Colab
# vacuum_problem = GraphProblemNonDet('DDL', set(['CCL', 'CCR']), vacuum_world)

### Question
Return a dictionary that defines a Graph representing transitions in the three room erratic vacuum world.
    Use letter `M` to denote that the agent is in the middle room.
    For example, `CDCM` means `the left room and right rooms are clean, the middle room is dirty; the robot is in the middle room`.

For reference, our solution is **27** line(s) of code.

In [ ]:
def three_room_erratic_world():
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def three_room_erratic_world_test_partial():
    """Checks if `three_room_erratic_world` returns a correct partial answer.

    A full check will be done by comparing the MD5 of the returned
    result, between the submission and our solution.
    """
    ans = three_room_erratic_world()
    assert isinstance(ans, dict), "Incorrect return type"
    ans = sort_graph_dict(ans)
    partial_result = sort_graph_dict(
        dict(
            DDDL=dict(Suck=['CDDL', 'CCDL'], Right=['DDDM']),
            DDDM=dict(Suck=['DCDM', 'CCDM', 'DCCM'],
                      Right=['DDDR'],
                      Left=['DDDL']),
            DDDR=dict(Suck=['DDCR', 'DCCR'], Left=['DDDM']),
            DDCL=dict(Suck=['CDCL', 'CCCL'], Right=['DDCM']),
            DDCM=dict(Suck=['DCCM', 'CCCM'], Right=['DDCR'], Left=['DDCL']),
            DDCR=dict(Suck=['DDCR', 'DDDR'], Left=['DDCM']),
            DCDL=dict(Suck=['CCDL'], Right=['DCDM']),
            DCDM=dict(Suck=['DCDM', 'DDDM'], Right=['DCDR'], Left=['DCDL']),
            DCDR=dict(Suck=['DCCR'], Left=['DCDM']),
            DCCL=dict(Suck=['CCCL'], Right=['DCCM']),
        ))
    for k in partial_result:
        assert k in ans, f"answer should have '{k}' as a key"
    assert {k: ans[k] for k in partial_result
           } == partial_result, "submission does not match the partial result"

three_room_erratic_world_test_partial()



assert graph_dict_md5(three_room_erratic_world()) == "0434567000d839f48149a9d73c8c4a70"
print('Tests passed.')

## Warmup


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


def lit_to_var_val(literal):
    """Converts a literal into (variable, value).

    Args:
      literal: A nonzero int.

    Returns:
      variable: A positive int representing a variable.
      value: True or False, i.e., positive or negative.
    """
    return abs(literal), literal > 0


def is_cnf_formula(formula):
    """Checks whether the input is a valid CNF formula.

    A formula is in valid CNF form if it is a list of lists of nonzero
    integers with sign indicating whether the variable is negated.

    You will not need to use this utility in your implementation,
    but it may be useful to read to understand the CNF representation.
    """
    if not isinstance(formula, list):
        return False
    if len(formula) == 0:
        return True
    clause = formula[0]
    if not isinstance(clause, list):
        return False
    for literal in clause:
        if not isinstance(literal, int):
            return False
        if literal == 0:
            return False
    if len(formula) == 1:
        return True
    return is_cnf_formula(formula[1:])


def get_variables_in_cnf_formula(cnf_formula):
    """Get a list of all variables in a CNF formula.

    Args:
      cnf_formula: A list of lists of nonzero ints.

    Returns:
      variables: A list of all variables that appear in
        the formula.
    """
    variables = set()
    for clause in cnf_formula:
        variables.update({lit_to_var_val(literal)[0] for literal in clause})
    variables = sorted(variables)
    return variables

### Question
In this problem, CNF formulas are represented as lists of lists of nonzero integers. The sign of the integer represents whether the corresponding proposition is negated or not. For example, the formula ((x1 or not x2) and (x3 or x2)) would be represented as [[1, -2], [3, 2]]. Complete the following function to confirm your understanding of this representation.

For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup():
  """Return a list of lists of ints for the CNF formula:

  ((x4 or not x5 or not x6) and (x6 or x5 or not x1) and (x2 or x3)).

  Keep the same order as in the formula above.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
# Note: the catsoop test is more strict for this question
# than the unit tests here. Make sure that your answer
# matches the description in the docstring exactly.
assert is_cnf_formula(warmup())
assert get_variables_in_cnf_formula(warmup()) == [1, 2, 3, 4, 5, 6]

print('Tests passed.')

## DPLL


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


def clause_is_determined(clause, partial_assignment):
    """Checks whether all variables in the clause have an assignment.

    Args:
      clause: A list of nonzero ints.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_determined: True if all variables in the clause appear in
        partial_assignment.
    """
    for literal in clause:
        if not (literal in partial_assignment or
                -literal in partial_assignment):
            return False
    return True


def literal_is_satisfied(literal, partial_assignment):
    """Checks whether the literal is satisfied by the assignment.

    Args:
      literal: A nonzero int.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_satisfied: True if the literal's variable appears in the
        partial_assignment, with a sign matching the literal.
    """
    variable, val = lit_to_var_val(literal)
    return variable in partial_assignment and partial_assignment[variable] == val


def clause_is_satisfied(clause, partial_assignment):
    """Checks whether the clause is satisfied by the assignment.

    Args:
      clause: A list of nonzero ints.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_satisfied: True if all the literals in the clause are
        satisfied.
    """
    for literal in clause:
        if literal_is_satisfied(literal, partial_assignment):
            return True
    return False


def find_pure_variable(cnf_formula, variables, partial_assignment):
    """Helper for DPLL.

    A variable is pure if it has the same sign in all unsatisfied clauses
    and if it is not already assigned.

    If a pure variable exists, this function returns the variable and value
    corresponding to the literal. (If multiple exist, return an arbitrary one.)

    If no pure variables exist, return (None, None).

    Args:
      cnf_formula: A list of lists of nonzero integers representing a CNF formula,
        with sign indicating whether the variable is negated.
      variables: A list of positive integers.
      partial_assignment: A dict mapping positive integers to bools, or None for
        an empty assignment.

    Returns:
      variable : A positive integer or None.
      value: A bool or None.
    """
    candidate_to_possible_values  = {v : {True, False} for v in variables \
                                    if v not in partial_assignment}
    for clause in cnf_formula:
        if clause_is_satisfied(clause, partial_assignment):
            continue
        for literal in clause:
            variable, value = lit_to_var_val(literal)
            if variable in candidate_to_possible_values:
                candidate_to_possible_values[variable].discard(not value)
    for candidate, possible_values in candidate_to_possible_values.items():
        if possible_values:
            value = next(iter(possible_values))
            return candidate, value
    return None, None


def find_unit_clause(cnf_formula, partial_assignment):
    """Helper for DPLL.

    A clause is a unit clause if all literals but one are already assigned to
    False. If a unit clause exists, this function returns the variable and value
    corresponding to the literal. (If multiple exist, return an arbitrary one.)

    Args:
      cnf_formula: A list of lists of nonzero integers representing a CNF formula,
        with sign indicating whether the variable is negated.
      variables: A list of positive integers.
      partial_assignment: A dict mapping positive integers to bools, or None for
        an empty assignment.

    Returns:
      variable : A positive integer or None.
      value: A bool or None.
    """
    for clause in cnf_formula:
        unassigned_literal = None
        is_unit_clause = True
        for literal in clause:
            # If the literal is true in the assignment, this is not a unit clause
            if literal_is_satisfied(literal, partial_assignment):
                is_unit_clause = False
                break
            # If the literal is false in the assignment, this could be a unit clause
            elif literal in partial_assignment:
                continue
            # If there is already an unassigned literal, this is not a unit clause
            elif not (unassigned_literal is None):
                is_unit_clause = False
                break
            else:
                unassigned_literal = literal
        if is_unit_clause and not (unassigned_literal is None):
            return lit_to_var_val(unassigned_literal)
    return None, None

### Question
Use your helper functions to complete an implementation of DPLL.

For reference, our solution is **59** line(s) of code.

In [ ]:
def run_inference_dpll(cnf_formula):
  """Find a satisfying assignment for a propositional CNF formula with DPLL.

  Args:
    cnf_formula: A list of lists of nonzero integers representing a CNF formula,
      with sign indicating whether the variable is negated.

  Returns:
    satisfiable: A bool indicating whether some satisfying assignment exists.
    assignment: A dict mapping positive integers to bools, or None if no
      satisfying assignment exists.

  Examples:
    >> run_inference_dpll([[1, -2], [-1, -2]])
    >> (True, {1: True, 2: False}))

    >> run_inference_dpll([[1], [-1]])
    >> (False, None)
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert run_inference_dpll([]) == (True, {})


assert run_inference_dpll([[1]]) == (True, {1: True})


assert run_inference_dpll([[-1]]) == (True, {1: False})


assert run_inference_dpll([[1, 2]]) in [(True, {1: True, 2: True}), (True, {1: True, 2: False}), (True, {1: False, 2: True})]


assert run_inference_dpll([[-1, 2]]) in [(True, {1: True, 2: True}), (True, {1: False, 2: True}), (True, {1: False, 2: False})]


assert run_inference_dpll([[1], [-1]]) == (False, None)


assert run_inference_dpll([[1, 2, 3], [-1, -2, -3], [1, -2, 3], [-1], [-3]]) == (False, None)


assert run_inference_dpll([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]) == (True, {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True, 10: True, 11: True, 12: True, 13: True, 14: True, 15: True, 16: True, 17: True, 18: True, 19: True, 20: True, 21: True, 22: True, 23: True, 24: True, 25: True, 26: True, 27: True, 28: True, 29: True, 30: True, 31: True, 32: True})
print('Tests passed.')

## Propositional Sentence Evaluation


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import collections

## Common logic data structures
Not = collections.namedtuple("Not", ["sentence"])
And = collections.namedtuple("And", ["sentence1", "sentence2"])
Or = collections.namedtuple("Or", ["sentence1", "sentence2"])
Implies = collections.namedtuple("Implies", ["sentence1", "sentence2"])

## Propositional logic data structures
Proposition = str  # Name of the proposition
PropositionalModel = dict  # Proposition -> bool

## Example of PropositionalModel, used in tests
IS_RAINING = Proposition("is-raining")
IS_SUNNY = Proposition("is-sunny")
NEED_UMBRELLA = Proposition("need-umbrella")
PROP_MODEL = PropositionalModel({
    IS_RAINING: True,
    IS_SUNNY: False,
    NEED_UMBRELLA: True,
})

### Question
*Note: for these questions, refer to the top of the Colab notebook.*
Write a function that takes a propositional sentence and evaluates it against a single model.
You may find python's builtin `isinstance` useful. For example, `isinstance(sentence, And)` returns whether a sentence is an `And`.

For reference, our solution is **15** line(s) of code.

In [ ]:
def evaluate_propositional_sentence(sentence, model):
  """Evaluate a propositional sentence against a single model.

  Args:
    sentence: A Proposition, And, Or, Not, or Implies.
    model: A PropositionalModel.

  Returns:
    holds: A bool representing the truth value of the sentence
      under the model.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:
assert evaluate_propositional_sentence(IS_RAINING, PROP_MODEL) == True


assert evaluate_propositional_sentence(Not(IS_RAINING), PROP_MODEL) == False


assert evaluate_propositional_sentence(And(IS_RAINING, IS_SUNNY),
                                       PROP_MODEL) == False


assert evaluate_propositional_sentence(Implies(IS_SUNNY, Not(IS_RAINING)),
                                       PROP_MODEL) == True


assert evaluate_propositional_sentence(Or(IS_RAINING, Not(IS_RAINING)),
                                       PROP_MODEL) == True

print('Tests passed.')

## Sympy Warmup 1


### Question
Use sympy to determine whether the following formula is satisfiable:
$(\neg x_1 \land x_2) \Rightarrow ((x_2 \lor x_3) \land (x_1 \lor \neg x_3))$.

Note that the return type should be **bool**.


For reference, our solution is **4** line(s) of code.

In [ ]:
def formula1_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula1_is_satisfiable() == True
print('Tests passed.')

## Sympy Warmup 2


### Question
Use sympy to determine whether the following formula is satisfiable:
$(x_1 \lor x_2) \land (\neg x_1 \lor \neg x_2) \land (x_1 \lor \neg x_2) \land (\neg x_1 \lor x_2)$.


For reference, our solution is **4** line(s) of code.

In [ ]:
def formula2_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula2_is_satisfiable() == False
print('Tests passed.')

## Search and Rescue Inference


### Question
Write a program that takes a grid as input and infers unknown values.

Your program should output a new grid with all determinable unknown values replaced with the inferred value. If an unknown value cannot be determined, it should be left unknown.

**Your program should use sympy.**


For reference, our solution is **57** line(s) of code.

In [ ]:
def infer_unknown_values(grid):
  """Fill in any unknown values in the grid that can be inferred.

  Args:
    grid: A list of lists of "F", "U", "S", or "C".

  Returns:
    inferred_grid: A copy of grid with some unknown values replaced.

  Example:
    >> grid = [
    >>   ["F", "U", "C"],
    >>   ["S", "C", "U"],
    >>   ["U", "U", "C"]
    >> ]
    >> infer_unknown_values(grid)
    >> [["F" "S" "C"]
    >>  ["S" "C" "C"]
    >>  ["U" "U" "C"]]
  """
  raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert infer_unknown_values([["U", "F"]]) == [["S", "F"]]


assert infer_unknown_values([["F", "U", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["F", "S", "C"], ["S", "C", "C"], ["U", "U", "C"]]


assert infer_unknown_values([["U", "C", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["C", "C", "C"], ["S", "C", "C"], ["F", "S", "C"]]


assert infer_unknown_values([["U", "S", "C", "U"], ["U", "U", "C", "U"], ["U", "S", "C", "U"]]) == [["F", "S", "C", "C"], ["S", "C", "C", "C"], ["F", "S", "C", "C"]]


assert infer_unknown_values([["U", "U", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "F", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"]]) == [["C", "C", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "S", "U", "U", "U", "U"], ["U", "U", "S", "F", "S", "U", "U", "U"], ["U", "U", "U", "S", "U", "U", "U", "U"]]
print('Tests passed.')